# Handcalc-style Dynamic LaTeX with SymPy

This notebook shows a compact pattern to:
1. Define a symbolic expression with **SymPy**.
2. Display the symbolic formula in LaTeX.
3. Substitute numeric values and render that step in LaTeX.
4. Evaluate numerically and present the result in LaTeX.

> Tip: install SymPy if needed via `pip install sympy`.


In [ ]:

from sympy import symbols, sin, cos, Eq, sympify, latex
from sympy import Matrix, simplify
from IPython.display import display, Math

def _fmt_value(v, precision=6):
    """Format numbers cleanly for LaTeX output."""
    try:
        return latex(v.evalf(precision))
    except Exception:
        return latex(v)


In [ ]:

def handcalc(expr, values, symbol="y", precision=6):
    """
    Render a handcalc-style derivation with SymPy.
    
    Parameters
    ----------
    expr : sympy expression or str
        The symbolic expression (e.g., "a*b/c + sin(theta)").
    values : dict
        Mapping from sympy Symbols (or their names) to numeric values.
        Example: {a: 3, b: 4, c: 2, theta: 0.2}
    symbol : str
        Output variable symbol name used in the displayed equations.
    precision : int
        Number of significant digits to display in the numeric step.
    
    Returns
    -------
    numeric_value : sympy.Float
        The numerically evaluated result of the expression after substitution.
    """
    # Accept strings and convert to SymPy expressions when needed
    if isinstance(expr, str):
        expr = sympify(expr)
    
    # Allow keys in 'values' to be either Symbols or strings
    sym_keys = {}
    for k, v in values.items():
        if isinstance(k, str):
            sym_keys[sympify(k)] = v
        else:
            sym_keys[k] = v
    
    # Step 1: symbolic
    sym_ltx = latex(expr)
    display(Math(rf"{symbol} = {sym_ltx}"))
    
    # Step 2: substitution (symbolic expression with numbers filled in)
    substituted = expr.subs(sym_keys)
    sub_ltx = latex(substituted)
    
    # Show compact assignments line
    assignments = ",\; ".join([rf"{latex(k)}={_fmt_value(v, precision)}" for k, v in sym_keys.items()])
    if assignments:
        display(Math(r"\text{with } " + assignments))
    display(Math(rf"{symbol} = {sub_ltx}"))
    
    # Step 3: numeric evaluation
    numeric_value = substituted.evalf(precision)
    display(Math(rf"{symbol} \approx {latex(numeric_value)}"))
    return numeric_value


## Example 1 — Mixed algebraic + trigonometric expression

In [ ]:

a, b, c, theta = symbols('a b c theta', real=True)
expr = a*b/c + sin(theta)

vals = {a: 3.0, b: 4.0, c: 2.0, theta: 0.2}
_ = handcalc(expr, vals, symbol="z", precision=8)


## Example 2 — Kinematics (displacement)

In [ ]:

# s = v0 * t + 1/2 * a * t^2
v0, t, acc = symbols('v0 t acc', real=True)
s_expr = v0*t + (acc * t**2) / 2

vals2 = {v0: 12.5, acc: -9.81, t: 1.8}
_ = handcalc(s_expr, vals2, symbol="s", precision=6)


## Example 3 — Quadratic root (one branch of the formula)

In [ ]:

from sympy import sqrt
a_, b_, c_ = symbols('a_ b_ c_', real=True)
x_expr = (-b_ + sqrt(b_**2 - 4*a_*c_)) / (2*a_)

vals3 = {a_: 1.0, b_: -5.0, c_: 6.0}  # roots should be 2 and 3
_ = handcalc(x_expr, vals3, symbol="x", precision=6)
